In [ ]:
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import time

In [ ]:
import re

def transform_date_to_month_count(date_str):
    match = re.match(r'(\d+)年(\d+)月', date_str)
    if match:
        year, month = map(int, match.groups())
        return (year - 1) * 12 + month
    else:
        return None

def get_twse_dividend_history(start_date: str, end_date: str):
    url = f"https://www.twse.com.tw/exchangeReport/TWT49U?response=html&strDate={start_date}&endDate={end_date}"
    dividend_history = pd.read_html(url)

    return dividend_history[0]
global div_all_data
div_all_data = get_twse_dividend_history("20120101", "20220101")

In [ ]:
global stock_all_data
global div_codes
div_codes = {}
stock_all_data = pd.DataFrame(columns=["Date", "open", "high", "low", "close", "volumn"])

In [ ]:
def future_new(tp, code):
    global stock_all_data
    if code in stock_all_data.columns:
        return stock_all_data[code]
    print("正在抓取", code, "的資料。")
    res = requests.get('https://tw.quote.finance.yahoo.net/quote/q?type=ta&perd=' + str(tp) + '&mkt=10&sym=' + str(code) + '&v=1&callback=jQuery111308938933339791952_1703128480417&_=1703128480418')
    m = re.findall('{"t":(\d+),"o":([\d.]+),"h":([\d.]+),"l":([\d.]+),"c":([\d.]+),"v":([\d.]+)}', res.text)
    res_df = pd.DataFrame(m)
    res_df.columns = ['Date', 'open', 'high', 'low', 'close', "volumn"]
    # res_df['Date'] = pd.to_datetime(res_df['Date'], format = '%Y%m%d%H%M')
    res_df = res_df.set_index("Date")
    convert_dict = {"open":float, "high":float, "low":float, "close":float, "volumn":float}
    res_df = res_df.astype(convert_dict)
    stock_all_data[code] = res_df["close"]

    return res_df["close"]

def expected_return_and_std(code, stock, shift):
    global div_all_data
    global div_codes
    stock_shift = stock.shift(shift)
    stock = stock[shift:]
    stock_shift = stock_shift[shift:]
    stock_return = (stock - stock_shift) / stock_shift
    code1 = div_all_data[div_all_data["股票代號"] == str(code)]
    month = [transform_date_to_month_count(i) for i in code1['資料日期']]
    if str(code) in div_codes:
        mean_div = div_codes[str(code)]
    else:
        div_month_return = []
        for i in range(1, len(month)):
            div_month_return.append( (1 + (code1["權值+息值"].iloc[i] / code1['除權息參考價'].iloc[i-1])) ** (12/(month[i] - month[i-1])) - 1)
        if not div_month_return:
          return np.mean(stock_return), stock_return[::-1]
        mean_div = np.mean(div_month_return)
        div_codes[str(code)] = mean_div
    # print(code, " mean_div: ", mean_div)
    stock_return = stock_return + mean_div
    expected_return = np.mean(stock_return) + mean_div
    return expected_return, stock_return[::-1]

In [ ]:
import numpy as np
from scipy.optimize import minimize

def max_return(expected_returns, cov_matrix, sigma_max, names):
    risk_free_rate = 0.016    # Risk-free rate
    # Number of stocks
    n = len(expected_returns)  # Get the length of stocks

    # Define the objective function (negative expected return)
    def objective(x):
        return -np.dot(expected_returns, x)

    # Define the constraint for standard deviation
    def constraint(x):
        std_portfolio = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
        return sigma_max - std_portfolio

    # Constraints and bounds
    cons = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of proportions equals 1
            {'type': 'ineq', 'fun': constraint}]               # Standard deviation constraint

    bounds = [(0, 1)] * n  # Proportions between 0 and 1 for each stock

    # Initial guess
    x0 = np.array([1/n] * n)

    # Solve
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
    print("Success:", result.success)
    # Results
    optimal_proportions = result.x
    max_return = -(result.fun)
    min_volatility = np.sqrt(np.dot(optimal_proportions.T, np.dot(cov_matrix, optimal_proportions)))
    max_sharpe_ratio = (max_return - risk_free_rate) / min_volatility

    for i, name in enumerate(names):
        print("Optimal proportions of ", name, ":", optimal_proportions[i])
    print("Maximum Sharpe Ratio:", round(max_sharpe_ratio, 5))
    print(f"Maximum expected return: {round(max_return, 5)}")
    print(f"Minimum volatility: {round(min_volatility, 5)}")

In [ ]:
import numpy as np
from scipy.optimize import minimize

def max_sharpe(expected_returns, cov_matrix, sigma_max, expected_return_max, names):
    risk_free_rate = 0.016    # Risk-free rate
    n = len(expected_returns) # Get the length of stocks

    # Define the negative Sharpe ratio (since we're minimizing)
    def negative_sharpe_ratio(x):
        Rp = np.dot(x, expected_returns)
        sigma_p = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
        return -(Rp - risk_free_rate) / sigma_p

    def constraint_std(x):
        std_portfolio = np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))
        return sigma_max - std_portfolio

    def constraint_e(x):
        e = np.dot(expected_returns, x)
        return e - expected_return_max

    cons = [] # Constraints and bounds
    cons.append({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    cons.append({'type': 'ineq', 'fun': constraint_e})
    if sigma_max != 0:
        cons.append({'type': 'ineq', 'fun': constraint_std})
    bounds = tuple((0, 1) for _ in expected_returns)

    initial_guess = np.array([1 / n] * n) # Initial guess

    # Perform the optimization
    result = minimize(negative_sharpe_ratio, initial_guess, method='SLSQP', bounds=bounds, constraints=cons)
    print("Success:", result.success)
    # Results
    optimal_proportions = result.x
    max_sharpe_ratio = -result.fun
    max_return = np.dot(expected_returns, optimal_proportions)
    min_volatility = np.sqrt(np.dot(optimal_proportions.T, np.dot(cov_matrix, optimal_proportions)))

    for i, name in enumerate(names):
        print("Optimal proportions of ", name, ":", optimal_proportions[i])
    print("Maximum Sharpe Ratio:", round(max_sharpe_ratio, 5))
    print(f"Maximum expected return: {round(max_return, 5)}")
    print(f"Minimum volatility: {round(min_volatility, 5)}")

In [ ]:
def min_volatility(expected_returns, cov_matrix, expected_return_max, names):
    risk_free_rate = 0.016    # Risk-free rate
    # Number of stocks
    n = len(expected_returns)  # Get the length of stocks

    # Define the objective function (negative expected return)
    def objective(x):
        return np.sqrt(np.dot(x.T, np.dot(cov_matrix, x)))

    # Define the constraint for standard deviation
    def constraint(x):
        e = np.dot(expected_returns, x)
        return e - expected_return_max

    # Constraints and bounds
    cons = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of proportions equals 1
            {'type': 'ineq', 'fun': constraint}]               # Standard deviation constraint

    bounds = [(0, 1)] * n  # Proportions between 0 and 1 for each stock

    # Initial guess
    x0 = np.array([1/n] * n)

    # Solve
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
    print("Success:", result.success)

    # Results
    optimal_proportions = result.x
    max_return = np.dot(expected_returns, optimal_proportions)
    min_volatility = result.fun
    max_sharpe_ratio = (max_return - risk_free_rate) / min_volatility

    for i, name in enumerate(names):
        print("Optimal proportions of ", name, ":", optimal_proportions[i])
    print("Maximum Sharpe Ratio:", round(max_sharpe_ratio, 5))
    print(f"Maximum expected return: {round(max_return, 5)}")
    print(f"Minimum volatility: {round(min_volatility, 5)}")

In [ ]:
def main_part(stock1 = 0, stock2 = 0, stock3 = 0, stock4 = 0, stock5 = 0, op_type = "MaxSharpe", sigma_max = 0, expected_return_max = 0.016):
    expected_return_max = max(expected_return_max, 0.016)
    # print(expected_return_max)
    # print(sigma_max)
    codes = [stock1, stock2, stock3, stock4, stock5]
    stocks = pd.DataFrame()
    es = []
    stock_returns = pd.DataFrame()
    # stds = []
    for i in codes:
        if i != 0:
            stocks[i] = future_new("m", i)
            # print(stocks)
            es.append(expected_return_and_std(i, stocks[i], 12)[0])
            stock_returns[i] = (expected_return_and_std(i, stocks[i], 12)[1])
            # stds.append(expected_return_and_std(stocks[i], 1)[1])
    covs = stock_returns.cov()
    # print(covs)
    if op_type == "MaxSharpe":
        return max_sharpe(es, covs, sigma_max, expected_return_max, stock_returns.columns)
    elif op_type == "MaxReturn":
        return max_return(es, covs, sigma_max, stock_returns.columns)
    elif op_type == "MinVolatility":
        return min_volatility(es, covs, expected_return_max, stock_returns.columns)
    else:
        print("Invalid op_type")

In [ ]:
main_part(stock1 = "00715L", stock2 = "00687B", stock3 = "0050", stock4 = "2330", stock5 = "00632R", op_type = "MaxSharpe", sigma_max = 0, expected_return_max = 0.05)

正在抓取 00632R 的資料。
Success: True
Optimal proportions of  00715L : 0.14703652667574973
Optimal proportions of  00687B : 0.0
Optimal proportions of  0050 : 3.0899761915836876e-18
Optimal proportions of  2330 : 0.8529634733242504
Optimal proportions of  00632R : 0.0
Maximum Sharpe Ratio: 0.86096
Maximum expected return: 0.27267
Minimum volatility: 0.29812


In [ ]:
from ipywidgets import interact_manual
interact_manual(
    main_part,
    stock1 = "00715L",
    stock2 = "00687B",
    stock3 = "0050",
    stock4 = "2330",
    stock5 = "00632R",
    op_type = ["MaxSharpe", "MaxReturn", "MinVolatility"],
    sigma_max = (0, 0.5, 0.01),
    expected_return_max = (0.05, 0.5, 0.01)
);

interactive(children=(Text(value='00715L', description='stock1'), Text(value='00687B', description='stock2'), …